# RAG LLM Quick Start Notebook

This notebook demonstrates how to use the RAG LLM system.

In [ ]:
import sys
sys.path.insert(0, '../src')

from rag_llm.ingestion import DocumentLoader, TextSplitter
from rag_llm.embeddings import EmbeddingModel, VectorStore
from rag_llm.retrieval import Retriever
from rag_llm.llm import LLMClient
from rag_llm.query_processor import QueryProcessor

## 1. Load Documents

In [ ]:
# Load documents from directory
loader = DocumentLoader('../data/documents')
documents = loader.load()
print(f"Loaded {len(documents)} documents")

## 2. Split into Chunks

In [ ]:
# Split documents
splitter = TextSplitter(chunk_size=1000, chunk_overlap=200)
chunks = splitter.split_documents(documents)
print(f"Created {len(chunks)} chunks")

## 3. Generate Embeddings

In [ ]:
# Generate embeddings
embedding_model = EmbeddingModel()
embeddings = embedding_model.embed_texts([chunk['content'] for chunk in chunks])
print(f"Generated embeddings with shape: {embeddings.shape}")

## 4. Build Vector Store

In [ ]:
# Build and save vector store
vector_store = VectorStore()
vector_store.add_documents(chunks, embeddings)
vector_store.save()
print("Vector store built and saved")

## 5. Query the System

In [ ]:
# Initialize components
retriever = Retriever(vector_store, embedding_model)
llm_client = LLMClient(model_name='gpt-3.5-turbo', provider='openai')
processor = QueryProcessor(retriever, llm_client)

# Process a query
query = "What is RAG?"
result = processor.process_query(query)

print(f"Query: {result['query']}")
print(f"\nAnswer: {result['answer']}")
print(f"\nSources: {len(result['sources'])} documents")